<a href="https://colab.research.google.com/github/subhash1208/Amazon-ML-Challenge-Team-AutoGen-/blob/main/Team_AutoGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Cell 1: Import Required Libraries
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from src.utils import download_images  # Assuming this function is defined in utils.py
from src.constants import entity_unit_map, allowed_units  # Importing the unit map
import cv2  # For image processing
import matplotlib.pyplot as plt  # For visualization
import torch  # For BERT and PyTorch
from transformers import BertTokenizer, BertForTokenClassification  # For BERT model
from transformers import Trainer, TrainingArguments  # For training the model
import random
import re
from transformers import EarlyStoppingCallback  # For callbacks


ModuleNotFoundError: No module named 'constants'

In [ ]:
# Cell 2: Load and Clean the Dataset
# Load the training and test datasets
train_df = pd.read_csv('dataset/train.csv')
test_df = pd.read_csv('dataset/test.csv')

# Display total null values in the dataset
print("Total null values in training dataset:")
print(train_df.isnull().sum())

# Replace null values in entity_value with a specified string
train_df['entity_value'].fillna('unknown', inplace=True)  # Replace NaN in entity_value with 'unknown'


In [ ]:
# Cell 3: Download Images
# Download images for both train and test datasets
download_images(train_df['image_link'].tolist(), download_folder='dataset/train_images')
download_images(test_df['image_link'].tolist(), download_folder='dataset/test_images')


In [ ]:
# Cell 4: Display Random Images with Entity Name and Value
# Display 5 random images with their entity names and values
def display_random_images(df, num_images=5):
    random_samples = df.sample(num_images)
    plt.figure(figsize=(15, 10))
    
    for i, (index, row) in enumerate(random_samples.iterrows()):
        img_path = os.path.join('dataset/train_images', row['image_link'].split('/')[-1])
        img = plt.imread(img_path)
        plt.subplot(1, num_images, i + 1)
        plt.imshow(img)
        plt.title(f"{row['entity_name']}: {row['entity_value']}")
        plt.axis('off')
    
    plt.show()

display_random_images(train_df, num_images=5)


In [ ]:
# Cell 5: Group Data by Entity Name
# Group the training data by entity_name
grouped_train_df = train_df.groupby('entity_name')

# Display the grouped data (optional)
for name, group in grouped_train_df:
    print(f"Entity Name: {name}, Number of Samples: {len(group)}")


In [ ]:
# Cell 6: Prepare Data for Model Training
# Create a mapping for allowed units and their abbreviations
unit_abbreviations = {
    # For 'item_weight' and 'maximum_weight_recommendation'
    'gram': ['g', 'gr', 'gm', 'grams', 'grm'],
    'kilogram': ['kg', 'kilograms', 'kgs'],
    'milligram': ['mg', 'milligrams', 'mgs'],
    'microgram': ['µg', 'mcg', 'micrograms'],
    'ounce': ['oz', 'ounces', 'ozs'],
    'pound': ['lb', 'lbs', 'pounds'],
    'ton': ['t', 'tons', 'tonne', 'tonnes'],

    # For 'item_volume'
    'millilitre': ['ml', 'milliliters', 'millilitres'],
    'litre': ['l', 'lit', 'liters', 'litres'],
    'cubic_centimetre': ['cc', 'cm³', 'cubic centimeters', 'cubic centimetres'],
    'cubic_metre': ['m³', 'cubic meters', 'cubic metres'],
    'gallon': ['gal', 'gallons'],
    'quart': ['qt', 'quarts'],
    'pint': ['pt', 'pints'],
    'cup': ['c', 'cups'],

    # For 'voltage'
    'volt': ['v', 'volts'],
    'kilovolt': ['kv', 'kilovolts'],
    'millivolt': ['mv', 'millivolts'],

    # For 'wattage'
    'watt': ['w', 'watts'],
    'kilowatt': ['kw', 'kilowatts'],
    'megawatt': ['mw', 'megawatts'],
    'gigawatt': ['gw', 'gigawatts'],

    # For 'height', 'depth', and 'width'
    'millimetre': ['mm', 'millimeters', 'millimetres'],
    'centimetre': ['cm', 'centimeters', 'centimetres'],
    'metre': ['m', 'meters', 'metres'],
    'kilometre': ['km', 'kilometers', 'kilometres'],
    'inch': ['in', 'inches'],
    'foot': ['ft', 'feet'],
    'yard': ['yd', 'yards'],
    'mile': ['mi', 'miles'],

    # Other common units
    'degree_celsius': ['°C', 'C', 'degrees Celsius'],
    'degree_fahrenheit': ['°F', 'F', 'degrees Fahrenheit'],
    'calorie': ['cal', 'calories'],
    'kilocalorie': ['kcal', 'kcals'],
    'joule': ['j', 'joules'],
    'pascal': ['Pa', 'pascals'],
    'bar': ['bar', 'bars'],
    'psi': ['psi', 'pounds per square inch'],
    'newton': ['N', 'newtons'],
    'fluid_ounce': ['fl oz', 'fluid ounces'],
}

# Prepare the data for BERT encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_data(df):
    return tokenizer(df['entity_value'].tolist(), padding=True, truncation=True, return_tensors='pt')

train_encodings = encode_data(train_df)


In [ ]:
# Cell 7: Define the Model
# Load BERT model for token classification
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(entity_unit_map))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
)

# Create a Trainer instance with EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,  # You will need to create a proper dataset class
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop training if no improvement
)


In [ ]:
# Cell 8: Train the Model
# Train the model
trainer.train()


In [ ]:
# Cell 9: Evaluate the Model
# Evaluate the model on the training set
train_predictions = trainer.predict(train_encodings)
train_preds = np.argmax(train_predictions.predictions, axis=2)

# Generate classification report
print(classification_report(train_df['entity_value'], train_preds))


In [ ]:
# Cell 10: Make Predictions on Test Set
# Prepare test data for predictions
test_encodings = encode_data(test_df)

# Make predictions
test_predictions = trainer.predict(test_encodings)
test_preds = np.argmax(test_predictions.predictions, axis=2)

# Map predictions back to entity names
test_df['prediction'] = [entity_unit_map[entity][pred] for entity, pred in zip(test_df['entity_name'], test_preds)]


In [ ]:
# Cell 11: Create Submission File
# Create a DataFrame for submission
submission_df = pd.DataFrame({
    'index': test_df['index'],
    'prediction': test_df['prediction']
})

# Save the DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' created successfully.")
